In [ ]:
%cd /usr/local/
!cd /usr/local/
!ls

/usr/local
bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:

!rm -rf cuda
!ln -s /usr/local/cuda-12.2 /usr/local/cuda

In [ ]:
!stat cuda

  File: cuda -> /usr/local/cuda-12.2
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 37h/55d	Inode: 3932509     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2024-05-29 17:52:44.276782276 +0000
Modify: 2024-05-29 17:52:44.155770472 +0000
Change: 2024-05-29 17:52:44.155770472 +0000
 Birth: 2024-05-29 17:52:44.155770472 +0000


Merge Sort
CPU and GPU

In [ ]:
%%writefile reduction.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <cuda.h>
#include <iostream>

#define MAX_THREADS_PER_BLOCK 1024

void bitonicSortCPU(int* array, int length)
{
    for (int k = 2; k <= length; k *= 2)
    {
        for (int j = k / 2; j > 0; j /= 2)
        {
            for (int i = 0; i < length; i++)
            {
                int ixj = i ^ j;

                if (ixj > i)
                {
                    if ((i & k) == 0)
                    {
                        if (array[i] > array[ixj])
                        {
                            std::swap(array[i], array[ixj]);
                        }
                    }
                    else
                    {
                        if (array[i] < array[ixj])
                        {
                            std::swap(array[i], array[ixj]);
                        }
                    }
                }
            }
        }
    }
}

__global__ void bitonicSortGPU(int* array, int j, int k)
{
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x;
    unsigned int ixj = i ^ j;

    if (ixj > i)
    {
        if ((i & k) == 0)
        {
            if (array[i] > array[ixj])
            {
                int temp = array[i];
                array[i] = array[ixj];
                array[ixj] = temp;
            }
        }
        else
        {
            if (array[i] < array[ixj])
            {
                int temp = array[i];
                array[i] = array[ixj];
                array[ixj] = temp;
            }
        }
    }
}

void displayArray(int* array, int size)
{
    for (int i = 0; i < size; ++i)
        std::cout << array[i] << " ";
    std::cout << std::endl;
}

int main()
{
    int arraySize = 1 << 20;
    int* hostArray = new int[arraySize];
    int* hostArrayCopy = new int[arraySize];

    int* deviceArray;
    int* deviceTempArray;

    // Initialize the array with random values
    srand(static_cast<unsigned int>(time(nullptr)));
    for (int i = 0; i < arraySize; ++i)
    {
        hostArray[i] = rand() % 100;
        hostArrayCopy[i] = hostArray[i];
    }

    // Print unsorted array
    std::cout << "\n\nUnsorted array: ";
    if (arraySize <= 100)
    {
        displayArray(hostArray, arraySize);
    }
    else
    {
        std::cout << "\nArray too large to print.\n";
    }

    // Allocate memory on GPU
    cudaMalloc((void**)&deviceTempArray, arraySize * sizeof(int));
    cudaMalloc((void**)&deviceArray, arraySize * sizeof(int));

    // Copy the input array to GPU memory
    cudaMemcpy(deviceArray, hostArray, arraySize * sizeof(int), cudaMemcpyHostToDevice);

    // Perform GPU bitonic sort and measure time
    cudaEvent_t startGPU, stopGPU;
    cudaEventCreate(&startGPU);
    cudaEventCreate(&stopGPU);
    float gpuElapsedTime = 0;

    // Initialize CPU clock counters
    clock_t startCPU, endCPU;

    // Set number of threads and blocks for kernel calls
    int threadsPerBlock = MAX_THREADS_PER_BLOCK;
    int blocksPerGrid = (arraySize + threadsPerBlock - 1) / threadsPerBlock;

    // Time the run and call GPU Bitonic Kernel
    cudaEventRecord(startGPU);
    for (int k = 2; k <= arraySize; k <<= 1)
    {
        for (int j = k >> 1; j > 0; j >>= 1)
        {
            bitonicSortGPU <<<blocksPerGrid, threadsPerBlock>>> (deviceArray, j, k);
        }
    }
    cudaEventRecord(stopGPU);

    // Transfer sorted array back to CPU
    cudaMemcpy(hostArray, deviceArray, arraySize * sizeof(int), cudaMemcpyDeviceToHost);
    cudaEventSynchronize(stopGPU);
    cudaEventElapsedTime(&gpuElapsedTime, startGPU, stopGPU);

    // Time the run and call CPU Bitonic Sort
    startCPU = clock();
    bitonicSortCPU(hostArrayCopy, arraySize);
    endCPU = clock();

    // Calculate elapsed CPU time
    double cpuElapsedTime = static_cast<double>(endCPU - startCPU) / (CLOCKS_PER_SEC / 1000.0);

    // Display sorted GPU array
    std::cout << "\n\nSorted GPU array: ";
    if (arraySize <= 100)
    {
        displayArray(hostArray, arraySize);
    }
    else
    {
        std::cout << "\nArray too large to print\n";
    }

    // Display sorted CPU array
    std::cout << "\nSorted CPU array: ";
    if (arraySize <= 100)
    {
        displayArray(hostArrayCopy, arraySize);
    }
    else
    {
        std::cout << "\nArray too large to print\n";
    }

    // Print the time of the runs
    std::cout << "\n\nGPU Time: " << gpuElapsedTime << " ms" << std::endl;
    std::cout << "\n\nCPU Time: " << cpuElapsedTime << " ms" << std::endl;

    // Clean up memory
    delete[] hostArray;
    delete[] hostArrayCopy;

    cudaFree(deviceArray);
    cudaFree(deviceTempArray);

    return 0;
}


Overwriting reduction.cu


In [ ]:
!nvcc -o reduction reduction.cu

In [ ]:
!./reduction




Unsorted array: 
Array too large to print.


Sorted GPU array: 
Array too large to print

Sorted CPU array: 
Array too large to print


GPU Time: 6.99309 ms


CPU Time: 1079.69 ms
